In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee
import pynsee.download

In [4]:
url = "https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert"
emissions = pd.read_csv(url)
emissions.head(2)

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207


In [5]:
secteurs = emissions.select_dtypes(include='number').columns
emissions["dep"] = emissions["INSEE commune"].str[:2]

In [6]:
from pynsee.download import download_file
filosofi = download_file("FILOSOFI_COM_2016")
filosofi.head(2)

Downloading: 100%|██████████| 2.67M/2.67M [00:22<00:00, 123kiB/s] 
Extracting: 100%|██████████| 10.6M/10.6M [00:00<00:00, 228MB/s]


,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,01001,L'Abergement-Clémenciat,313,795.5,22679,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,101,248,24382.083333333336,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
filosofi = (filosofi.astype({c: "float" for c in filosofi.columns[2:]}))

In [9]:
emissions_totales = pd.DataFrame(emissions.sum(numeric_only = True), columns = ["emissions"]).reset_index(names = "secteur")
emissions_totales['emissions (%)'] = 100*emissions_totales['emissions']/emissions_totales['emissions'].sum()
emissions_totales.sort_values("emissions", ascending = False).round()

,secteur,emissions,emissions (%)
8,Routier,126493164.0,24.0
0,Agriculture,87909694.0,17.0
6,Industrie hors-énergie,83573677.0,16.0
7,Résidentiel,63841398.0,12.0
3,CO2 biomasse hors-total,63519311.0,12.0
9,Tertiaire,39562729.0,7.0
5,Energie,22852034.0,4.0
2,Autres transports international,22238569.0,4.0
4,Déchets,14703580.0,3.0
1,Autres transports,6535446.0,1.0


In [21]:
import requests
from io import StringIO
import pandas as pd
cog_2023 = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv")
communes = cog_2023.loc[cog_2023['TYPECOM']=="COM"]
communes.loc[:, ['COM', 'DEP', 'REG']].nunique()

COM    34945
DEP      101
REG       18
dtype: int64

In [22]:
communes.groupby('DEP').agg({'COM': 'nunique'})

,COM
DEP,
01,392
02,798
03,317
04,198
05,162
...,...
971,32
972,34
973,22


In [20]:
communes.groupby('DEP').agg({'COM': 'nunique'}).reset_index().sort_values('COM', ascending = False)

,DEP,COM
62,62,890
1,02,798
80,80,772
57,57,725
76,76,708
...,...,...
96,971,32
99,974,24
98,973,22
100,976,17


In [24]:
filosofi["dep"] = filosofi["CODGEO"].str[:2]

In [25]:
# Implicite => il faut faire attention à l'ordre des opérations
filosofi.groupby('dep')['NBPERSMENFISC16'].sum()

dep
01     613088.0
02     514249.0
03     329435.0
04     156537.5
05     133992.5
        ...    
92    1583682.0
93    1586664.5
94    1345977.0
95    1226059.0
97    1191947.0
Name: NBPERSMENFISC16, Length: 97, dtype: float64

In [26]:
filosofi.groupby('dep').agg({'NBPERSMENFISC16': 'sum'})

,NBPERSMENFISC16
dep,
01,613088.0
02,514249.0
03,329435.0
04,156537.5
05,133992.5
...,...
92,1583682.0
93,1586664.5
94,1345977.0


# Jointure


In [27]:
emissions = emissions.reset_index(names = ['id_left'])
filosofi = filosofi.reset_index(names = ['id_right'])

In [28]:
inner_merged = emissions.merge(
  filosofi,
  left_on = ["INSEE commune", "dep"],
  right_on = ["CODGEO", "dep"],
  how = "inner"
)
inner_merged.head(3)

,id_left,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,...,27.0,8.8,6.9,2.8,2.1,2.0,-15.7,10457.083333,33880.555556,3.239962


In [29]:
left_merged = emissions.merge(
  filosofi,
  left_on = ["INSEE commune", "dep"],
  right_on = ["CODGEO", "dep"],
  how = "left"
)
left_merged.head(3)

,id_left,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,...,27.0,8.8,6.9,2.8,2.1,2.0,-15.7,10457.083333,33880.555556,3.239962
